# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print("網頁更新中...")
    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print("到達頁面底端")
        break

    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [6]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 國壽網路投保大噴發　累積保費收入破10億創保險史新猷

▲國泰人壽攜手2大旅遊平台Agoda、KLOOK，共同打造旅遊生態圈。（圖／國泰人壽提供）
記者紀佳妘／台北報導
國泰金（2882）旗下國泰人壽網路投保今年大噴發，除了保費收入大幅成長逾300%，更在註冊會員數、投保件數、累積保費三大網路投保指標均穩居業界冠軍，會員突破100萬人，市占率高達5成，累積保費收入爆量破10億。
近年來出國旅遊人次連年增長，不過根據統計顯示，僅約有41%的旅人會提早規劃個人旅遊保障，另有近4成的人則會在旅遊前夕或當天臨櫃投保，而有近19%的人疏漏了旅遊保障，因此國壽特別攜手Agoda及KLOOK等2大旅遊平台，一同打造旅遊生態圈，網路投保旅平險可享訂房、交通等優惠，藉此吸引民眾目光。
國泰人壽數位發展部協理林秉毅表示，網路投保旅平險連年成長關鍵秘訣在於，消費者最快1分鐘內即可完成投保，並一次取得意外醫療、海外突發疾病、30餘項急難救助以及獨家的海外醫療專機運送實物給付服務。
林秉毅指出，國壽也與國泰世華銀行合作，只要卡友滿額即加贈旅遊不便險保障，將旅遊途中可能出現的風險完整轉嫁，且在急難時提供強而有力的諮詢與救援服務，更希望無論任何狀況發生，旅人都能平安返家。
此外，國泰人壽從2014年推出網路投保旅平險以來，連年以最佳投保體驗與最完整保障獲封「旅人最愛」，每2個旅人就有超過1人選擇國泰人壽旅平險，主要是提供客戶同時享有旅平險、不便險、及專機險三重保障，讓旅人找出自己最適合的旅遊保障。
----------------------------------------------------------------------

[1] 200餘名台電工程師遭年末資遣　勞團要求「派遣歸零」

▲台灣勞動派遣產業工會今（26）日到行政院門口前進行記者會。（圖／記者余弦妙攝）
記者余弦妙／台北報導
台灣勞動派遣產業工會今（26）日到行政院門口前進行記者會，質疑政府當初喊要讓「派遣歸零」政策跳票，且今年底又有兩百餘名台電派遣工程師遭資遣，公部門根本是帶頭「假派遣、真僱傭」，因此提出兩大訴求希望政府重視。對此，行政院也回應，會依照行政程序7天內

▲ 凱基證券為慶祝中華開發金控60周年推出閃耀60百萬豪禮歡樂送活動。（圖／凱基證券提供）
記者李瑞瑾／綜合報導
凱基證券為慶祝中華開發金控60周年推出的【閃耀60百萬豪禮歡樂送】活動圓滿落幕，日前由凱基證券董事長許道義親自抽出Toyota百萬名車、幸福極光雙人遊的2位幸運得主。
凱基證券表示，此次大方回饋客戶的活動，受到客戶熱烈參與，客戶參與越多就有越多抽獎機會，一起爭取百萬元的豪華名車、極光雙人遊等好禮，活動期間為許多客戶進行理財規劃，共創造出近六萬個抽獎機會，成功打造出公司與客戶雙贏局面。
凱基證券致力發展以客戶需求為導向的投資理財服務，有計畫培訓專業顧問團隊，提供多元的海內外金融產品，除針對高資產客戶提供客製化商品，同時也用心規劃一般投資大眾的理財產品，隨時滿足不同客群需求的投資理財服務。例如：凱基證券提供的定期定額服務，台股可投資的股票檔數為同業中最多，更首創定期定額美股ETF服務，產品線從積極到保守、股票到債券、個股到產業，應有盡有，讓投資人擁有最多元的投資選擇。
新的一年即將來臨，凱基證券表示，將持續透過專業顧問團隊高素質的人力，結合專家團隊資源，在金融科技工具的輔助下，精準掌握客戶需求，提供與需求零距離的全方位理財服務，致力成為值得客戶信賴的理財好夥伴。
----------------------------------------------------------------------

[11] 東元小金雞東捷資訊上櫃　首日蜜月行情漲逾20%

▲東元集團會長黃茂雄(左五)、董事長邱純枝(左四)與東捷資訊董事長高尚偉(左三)帶領經營團隊進行上櫃掛牌敲鑼儀式。（圖／東捷資訊提供）
記者姚惠茹／台北報導
東元集團旗下小金雞東捷資訊（6697）今（26）日 以每股42.6元掛牌上櫃，早盤以50.6元開出，盤中最高一度來到51.8元，漲幅逾20%，隨後維持漲幅逾15%左右，順利展開蜜月行情。
東捷資訊主要是因應大數據、物聯網與AI崛起，運用原來在ERP整合規劃的製造業行業專家優勢，以及母公司東元集團累積的跨國企業數位化成功經驗，進而開拓AIoT衍生市場商機，其中在智慧工廠部分，已陸續取得汽車零組件、機械業等指標企業的採用，後勢看好。
東捷資訊表示，除了協助母公司東元電機全球馬達廠逐步蛻變為智慧工廠，讓交期平均縮短30%，產能利用率提升10%外，更協

記者李瑞瑾／台北報導
新台幣強勢無法擋，也讓日圓匯價維持在0.278徘徊，根據台銀今（26）日上午牌告的日圓現金賣出匯價，日圓來到0.2784元，改寫一年以來新低紀錄。
▲ 東京迪士尼樂園。（圖／記者蔡玟君攝）
新台幣不斷走強，也讓日圓匯價持續創新低。今天早上新台幣兌美元匯率一度強升逾1角來到30.095元兌1美元，是2018年6月15日（匯率30.002）以來新高；除了新台幣走強之外，日元匯率也因避險情緒驟降而走低，目前來到109.53日圓兌1美元，為12月初以來相對低點。
根據台銀今日上午牌告的現金賣出匯價，日圓來到0.2784元，是去（2018）年12月19日（匯價0.2776元）、一年以來的新低價位。
若以兌換新台幣5萬元來看，今天可換到17萬9,598日圓，相較8月13日的匯價0.3029，當時只能換到16萬5,071日圓，就相差了1萬4,527日圓，等於賺到了2張迪士尼門票。
----------------------------------------------------------------------

[22] 新台幣早盤強彈逾1角！　最高來到30.102元兌1美元

▲ 新台幣兌美元今天開盤貶值0.7分。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（26）天開盤貶值0.7分、以30.21兌1美元開出。
新台幣兌美元昨（25）日收盤收在30.203兌1美元，盤中並一度升破30.1元，央行更於尾盤進場調節收斂升勢；今天開盤則以30.21元開出、貶值0.7分。
開盤後新台幣也持續走揚，截至上午9點10分，新台幣為30.102元兌1美元，升值1.01角。
----------------------------------------------------------------------

[23] 促參成績單！台北蟬聯3年招商王　高雄成績大幅下滑

▲台北市長柯文哲。（圖／記者宋良義攝）
記者吳靜君／台北報導
財政部促參司公布今年民間參與公共建設成績，整體而言今年前11月的促參案件已經達到86件，簽約金額有892億元，若加計台北信義行政中心等招標案件，今年全年簽約案件可望突破千億元。根據統計，台北市前11月招商案件有10件，簽約金額已經破百億元，加計12月已經簽約的大案，今年妥妥拿下連續第3座招商王的寶座。
依據

圖、文／經濟部提供
經濟部12月19日指出台灣三大方案執行以來已成功吸引278家台商回台投資約新台幣8,332億元，創造超過67,601個國內就業機會，預計108年底到位金額達新台幣2,398億元。 
經濟部表示三大方案加速引導上中下游廠商回流，促成北部以「伺服器」與「網通設備」為主、中部以「自行車」及「工具機」產業為主、南部「汽車零組件」及「汽車電子」等六大產業供應鏈落地紮根。
蘇貞昌強調，諸多國際大企業也大舉來台投資，今年1到10月核准的僑外投資金額將近3千億元（美金98億元），較去年同期成長23%，顯示大家看好台灣，廠商用腳投票，資金「進來的多，出去的少」。  經濟部表示，「投資台灣三大方案」包括「歡迎台商回台2.0行動方案」、「根留台灣企業加速投資行動方案」及「中小企業加速投資行動方案」，其中「歡迎臺商回臺投資行動方案」有160家廠商通過審查，總投資金額超過新台幣7,097億元，預估可創造本國就業人數超過57,931人。
「根留臺灣企業加速投資行動方案」總投資金額超過新台幣850億元，預估可創造本國就業人數超過6,075人；「中小企業加速投資行動方案」總投資金額超過新台幣384億元，預估可創造本國就業人數超過3,595人。
（經濟部廣告）
----------------------------------------------------------------------

[35] 「亞洲最佳個人金融銀行」、「臺灣最佳銀行」四冠王　中國信託當仁不讓

▲中信金融園區。（圖／記者李毓康攝）
記者廖明慧／綜合報導
中國信託2019年重要海內外獎項一覽表
----------------------------------------------------------------------

[36] 試算給你看！基本生活費調高　有2子女就讀大學的四口之家...可扣抵稅額增1.6萬

▲財政部公告基本生活費為17.5萬元，對於有養小孩的四口之家，有達到省稅的效果（圖／123RF、示意圖）
記者吳靜君／台北報導
財政部今（25）日公布，今年度的每人基本生活費為17.5萬元，明年5月報稅可以適用，較去年增加4000元。依財政部估算，如果家裡有2名子女就讀大學的四口之家，基本生活費跟原有的扣除額相比，將有4.8萬元的扣除額度可用，若與前一年相比，扣抵金額再增

▲金管會主委顧立雄。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
部分民眾、中小企業與金融機構少有往來，或缺少信用資產擔保，想要取得銀行借款相當不容易。金管會主委顧立雄今（25）日表示，目前已請聯徵中心介接政府資訊平台，讓民眾取得銀行融資更有利；聯徵中心董事長郭建中也提及，目前規劃介接3項個人相關行為數據，預計明年1月上線，提供聯徵中心的會員進行查詢。
金管會主委顧立雄今天在第13屆金安獎及金質獎暨金優獎表揚大會受訪時指出，因應金融科技發展，金管會今年已核准設立3家純網銀，除了提高整體金融業競爭力外，聯徵中心也將隨著3家純網銀的加入，強化資料庫內容，讓信用體系更加完整。
顧立雄表示，為達到普惠金融政策目標及配合國家發展計畫，更指示由聯徵中心介接政府資訊平台，不僅能強化行為數據應用，更有利消費者取得銀行融資。
聯徵中心董事長郭建中也提及，配合政府推動金融科技與普惠金融政策，建置政府資料介接查調平台，協助會員線上查調公務機關資料，其中經濟部商業司的有限合夥與商業登記資料、國貿局出進口廠商登記資料則已納入聯徵中心資料庫建檔，已於今年3月26日上線。
至於介接個人相關行為數據部分，郭建中說，財政部個人財產及所得資料、勞保局職業投保資料及交通部駕籍與車籍資料共3項，預計在明年1月開放線上查詢。換句話說，民眾未來申請信用貸款、相關授信等服務時，銀行可透過聯徵中心的系統查詢相關資料，借款也能更為便利。
----------------------------------------------------------------------

[48] 遠見／找回孩子找對定位　地方創生再升級

文╱林讓均  攝影╱張智傑
當有一天台灣剩不到2000萬人，你會怎麼介紹台灣？
這個由《遠見》副總編輯邱莉燕拋出的問題，揭開第11場論壇的討論「地方創生的永續發展」。
國發會定調2019年是「台灣地方創生元年」，計畫從元月啟動，已上路近一年。經常到各地考察的國發會主委陳美伶，觀察在地方創生成功的方程式中，一定有「人」。她鼓勵中央級的公務員認養家鄉、回鄉服務，也期待各地方政府善用數位工具，例如AI、大數據，來發展地方特色產業鏈。
在竹山小鎮打造出全台灣最美麗民宿「天空的院子」，小鎮文創創辦人何培鈞原本是醫務背景，念大學時無意間在竹山發現一座百年三合院廢墟，人生從此不一樣。
他已走遍台灣、大陸活

▲ 美元匯價跌破30.4元。（圖／路透社）
記者李瑞瑾／台北報導
新台幣表現強勢，也讓美元匯價創下1年半以來新低。根據台銀今（25）天下午牌告的美元現金賣出匯價，美元為30.355元，相較今年8月初的高點匯價，換新台幣5萬如今可以多拿到79美元，等於多賺到了約新台幣2,400元。
新台幣兌美元匯率不斷走強，並在30.1元兌1美元附近震盪。今天開盤以小貶0.5分、30.21元兌1美元開出，隨後即轉為升值，午盤之後更大幅走揚，最高一來到30.08元兌1美元，升值1.25角。
另外，看到台銀牌告的現金賣出匯價，美元匯價來到30.355元，創下去年6月20日（匯價30.332元）、近1年半以來新低。
若想換新台幣5萬元，今天可以換到1647美元；但如是以8月5日匯價31.885元來計算，當時只能換到1568美元，相差了79美元，等於多賺到約新台幣2,400元。
----------------------------------------------------------------------

[56] 共信-KY肺癌新藥PTS302　申請中國大陸藥證進入最後階段

▲共信研發團隊。（圖／共信提供）
記者姚惠茹／台北報導
共信-KY（6617）今（25）日宣布，子公司紅日健達康(天津)公司通知，已將「對甲苯磺醯胺注射劑 (PTS302)」 的肺癌新藥資料，包括適應症使用說明書及上市後風險控制計畫書送達中國國家食品藥品監督管理局藥品審評中心 (CDE)，並已受理進入最後審核階段。
共信副總經理蕭斯欣表示，這代表CDE已經完成對肺癌新藥的審評工作，等到這階段通過之後，將改由「藥品認證管理中心」進行藥證核准前的最後階段「GMP 現場核查」，肺癌新藥 (PTS302)上市申請進度將順利向前推進。
蕭斯欣指出，由於中國大陸合作夥伴紅日藥業公司在生產肺癌新藥 (PTS302)上有多年的豐富經驗，因此共信-KY子公司紅日健達 康公司對日後的「GMP 現場核查」深具信心。
蕭斯欣強調，由於肺癌新藥 (PTS302)是以「與現有肺癌治療手段相比具有明顯治療優勢」而獲 CDE 列入優先審評審批中的一類新藥題目，也具有優先列入查廠的資格，日後若順利上市，將對公司營收帶來明顯挹注。
共信-KY表示，依據跨國 IQVIA 公司針對中國肺癌嚴重氣道阻塞病人數量調查及藥品的訂價推估， 肺癌新藥 

▲台經院院長張建一（左1）。（圖／記者余弦妙攝）
記者余弦妙／台北報導
台經院今（25）日公布最新景氣動向調查，台經院指出受美中貿易戰干擾未歇、全球經濟疲弱影響，市場需求保守觀望，讓傳統產業廠商出口、生產與接單表現依舊不振，但近期美中貿易緊張情勢暫告緩解，有助於提振市場信心，根據調查結果顯示，製造業營業氣候測驗點結束先前連兩個月下滑態勢轉為上揚。台經院院長張建一更表示，如果2020年沒有意外，台灣整體經濟成長率是可以樂觀看待。
台經院表示，近期美中貿易緊張情勢暫告緩解，有助於提振市場信心，使得一些權重較大的化學工業、鋼鐵基本工業、石油及煤製品業與金屬製品業廠商對當月與未來半年景氣看法樂觀的比例高於10月調查，且先進製程持續發威、記憶體市況已顯好轉、高階半導體封測競爭力突出、中國去美國化趨勢持續等支撐下，使得電子零組件業看好未來半年景氣的廠商比例明顯較10月調查為高。
據調查顯示，11月製造業營業氣候測驗點結束先前連兩個月下滑態勢轉為上揚，服務業營業氣候測驗點雖連四個月下滑，然跌幅漸縮小，營建業測驗點則上揚轉為下滑。製造業廠商認為當11月景氣為「好」比率為18.4%，較10月21.1%減少2.7個百分點，認為當月景氣為「壞」的比率則為27.6%，較10月34.7%減少7.1個百分點。
因此整體而言，11月製造業營業氣候測驗點為93.42點，較上月修正後之92.13點增加1.29點。
張建一表示，今年受惠於轉單效應影響，經濟成長率比預計的好很多，雖然整體經濟不壞，但值得注意的為產業間的差異非常大，尤其是半導體、電子業等高階製程的部分皆表現不錯，雖然中美貿易戰趨於緩和，但其實大多數廠商在中國投資信心非常薄弱，導致台灣在機械設備、金屬製品、化學材料等這幾個業別都不太好。
但張建一也說，台灣經濟整體而言是越來越好，但還是要重視產業間表現差異逐漸拉大的情況，如果中美貿易戰可以逐漸緩和，且中國可以履行第一階段的協議，讓美國滿意不再繼續加徵關稅，就會讓全球投資信心往上，這樣在2019年表現相對較差的產業，就機會可以表現得比今年好。
----------------------------------------------------------------------

[69] 中美局勢緩和、低利環境支撐　2020全球經濟成長將可反彈回升

▲ 施羅德投信上修明年經濟成長預測。

▲ 台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
迎接西方聖誕假期，華爾街交投清淡，美股四大指數漲跌互見；台股少了外資挹注持續量縮震盪，今（25）天開盤小漲2點、以11,978點開出。
截至上午9點04分，指數來到11,989點、上漲18點。
電子三雄中，台積電（2330）平盤以332元開出；鴻海（2317）開在91.1元、上漲0.22%；股王大立光（3008）以4900元開出、上漲0.61%。
美國股市昨（24）日於下午提前休市，投資人交易量減少，美股四大指數小漲小跌，其中那斯達克指數則是連續9天創新高。
道瓊指數下跌36.08點，收28515.45點、跌幅0.13%；那斯達克指數上漲7.24點，收8952.88點、漲幅0.08%；標普 500 指數下跌0.63點，收3223.38點、跌幅0.02%；費城半導體指數上漲5.08點，收1862.2點、漲幅0.27%。
----------------------------------------------------------------------

[79] 與住戶同樂！太平別墅新落成高規格舉辦聖誕晚會

【企劃特輯】
當天參加的住戶都很開心，甚至表示：「有幾次買房的經驗，也有住大樓與透天別墅的經驗，大樓花管理基金辦中秋晚會是很普遍的，住透天社區就沒遇過舉辦活動。會買生活家建設的房子除了房子很美建材很吸引我外，就是因為從頭到尾都跟以往的經驗不同，以往看房看了半天也不知道怎麼蓋的，但生活家建設在施工上介紹的非常詳細，不怕比較！因為以往的買房經驗讓我會特別注意一些問題，但生活家非常貼心，不僅都事先替我們想到而且做得非常好，還告訴我內行人才懂的差異性，讓我有驚豔到！住進來後對比感太強，知道建商有用心細節都顧到，住進來後真的很滿意！就連這次聖誕晚宴從邀請到參與活動，我真的沒遇過這麼用心又貼心的建商，餐點精緻好吃不馬虎，連活動都好玩，還有小提琴演奏，伴手禮跟抽獎禮物都非常高檔，整個質感跟一般大樓社區舉辦的截然不同！兒子還說像在住飯店！邀請朋友來家裡作客時，連朋友都非常喜歡我們家！想換房子的朋友都問我還有空屋嗎？想當我鄰居，我回答朋友不早說，我買完後介紹很多親朋好友來買，連新案預約都快預約完了啦！」
生活家建設在太平豪墅圈屢創第一！擁有29年優質營建口碑的「生活家建設」，是太平「泓大重劃區」第一、也是唯一的七期豪墅

目前台股在12000點左右盤整，許多投資人都期盼能盡早突破歷史高點12682。在《雲端最有錢》節目中，「獅公分析師」李永年提到當年12682點和這次12000點情況完全不一樣，上次一張股票甚至可以買超過一間房子，上攻類股主要是金融股、營建股、資產股，而這次主要是電子產業股。
李永年回顧當年股王國泰人壽，從1985年的每股40元起漲，最高價來到1975元，等於上漲49倍，如果在以當時每年配股5元計算，1985年買一張國泰人壽，1989年已經增加至5張，換算下來等於漲了246倍。對比目前股王大立光，從最低價漲到最高價，漲了40倍，歷時15年，反觀以前的國泰人壽僅花4年就漲了49倍，落差其實相當大。
的確，三位來賓都認為當年情況比現今還要好賺。不過當時股價一下跌，也讓不少投資人印堂發黑，李永年表示自己當時因為擔任證券公司投顧經理，市場敏感度高，所以早先一步逃跑。
李永年也提出一些當年股市崩盤前的徵兆，例如日本股市於1989年已反轉起跌，不過當時台股卻持續漲，這很明顯就怪異了；還有當時買股人氣沸騰，連出家人也瘋狂搶買，另外，當時主流股已露疲態，像是國泰人壽當時都不再創新高；還有，壓抑四年的四大基金（鴻運、國際...）展開飆升補漲行情。
但重點是台股目前是否有崩盤可能呢？李永年信心滿滿地表示，完全沒有崩盤徵兆，投資人可以放心。
其他人也看了這些～
►ETtoday 8歲歡慶！優惠好康都在這裡～～
----------------------------------------------------------------------

[91] 金管會揪金控幕後藏鏡人　大股東持股申報2020年7月全都露

▲金管會為揪出金控、銀行背後的藏鏡人，預計明年7月1日起開始申報。（圖／記者湯興漢攝）
記者紀佳妘／台北報導
金管會要揪出各金控、銀行背後的「藏鏡人」，要股權一切透明化，銀行局明（25）日發布申報應注意事項，要求申報實質受益人或最終控制人，並給予半年的緩衝期，將在明（2020）年7月起正式上路。
金管會銀行局完成修正「金融控股公司法第16條第2項持有已發行有表決權股份申報應注意事項」及「銀行法第25條第2項持有已發行有表決權股份申報應注意事項」，這次修正共有5大重點。
首先是參考「金融機構防制洗錢辦法」規定，增訂同一人、同一關係人或第三人為法人時，將實質受益人或最終控制權